In [ ]:
import os
arch = os.getenv("ARGS", "real")

In [ ]:
# PYTEST_XFAIL_AND_SKIP_NEXT: blocked by https://github.com/NGSolve/ngsPETSc/issues/17
assert arch == "real"

In [ ]:
try:
    import BACKEND_LIBRARY_IN  # noqa: F401
except ImportError:
    import ngsolve
else:
    try:
        import ngsolve
    except ImportError:
        if arch != "complex":
            !wget "https://PROJECT_NAME_IN.github.io/releases/ngsolve-install-real.sh" -O "/tmp/ngsolve-install.sh" && bash "/tmp/ngsolve-install.sh"
        else:
            !wget "https://PROJECT_NAME_IN.github.io/releases/ngsolve-install-complex.sh" -O "/tmp/ngsolve-install.sh" && bash "/tmp/ngsolve-install.sh"
        import ngsolve  # noqa: F401

In [ ]:
import numpy as np
import petsc4py
import petsc4py.PETSc
if arch != "complex":
    assert not np.issubdtype(petsc4py.PETSc.ScalarType, np.complexfloating)
else:
    assert np.issubdtype(petsc4py.PETSc.ScalarType, np.complexfloating)

In [ ]:
from mpi4py.MPI import COMM_WORLD
from ngsolve import *
from ngsPETSc import KrylovSolver

In [ ]:
mesh = Mesh(unit_square.GenerateMesh(maxh=0.5).Distribute(COMM_WORLD))

In [ ]:
fes = H1(mesh, order=1)

In [ ]:
u = fes.TrialFunction()
v = fes.TestFunction()

In [ ]:
a = BilinearForm(fes, symmetric=True)
a += u * v * dx
a.Assemble()

In [ ]:
gff = GridFunction(fes)
for i in range(fes.ndof):
    gff.vec[i] = i + 1

In [ ]:
F = LinearForm(fes)
F += gff * v * dx

In [ ]:
solver = KrylovSolver(a, fes, solverParameters={"ksp_type": "preonly", "pc_type": "lu"})
gfu = solver.solve(F)

In [ ]:
import numpy as np

In [ ]:
for i in range(fes.ndof):
    assert np.isclose(gfu.vec[i], i + 1)